# Урок 9. Языковое моделирование
## Задание
Разобраться с моделью генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = 'mcyri.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# Delete Unicode spescial symbol \ufeff
text = text.replace('\ufeff', '')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 24683 characters


In [4]:
print(text[:500])

Михаил Юрьевич Лермонтов 
Мцыри
     
Библиотека школьника – 
     
    Мцыри – на грузинском языке значит «неслужащий монах», нечто вроде «послушника». (Прим. Лермонтова.)
    
     
    Вкушая, вкусих мало меда, и се аз умираю.
    1-я Книга Царств
     
1
     
     Немного лет тому назад,
     Там, где, сливаяся, шумят,
     Обнявшись, будто две сестры,
     Струи Арагвы и Куры,
     Был монастырь. Из-за горы
     И нынче видит пешеход
     Столбы обрушенных ворот,
     И


In [5]:
text = text + text

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

85 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
text_as_int, text[:2000], len(text_as_int), len(text)

(array([37, 59, 72, ..., 19,  1,  0]),
 'Михаил Юрьевич Лермонтов \r\nМцыри\r\n     \r\nБиблиотека школьника – \r\n     \r\n    Мцыри – на грузинском языке значит «неслужащий монах», нечто вроде «послушника». (Прим. Лермонтова.)\r\n    \r\n     \r\n    Вкушая, вкусих мало меда, и се аз умираю.\r\n    1-я Книга Царств\r\n     \r\n1\r\n     \r\n     Немного лет тому назад,\r\n     Там, где, сливаяся, шумят,\r\n     Обнявшись, будто две сестры,\r\n     Струи Арагвы и Куры,\r\n     Был монастырь. Из-за горы\r\n     И нынче видит пешеход\r\n     Столбы обрушенных ворот,\r\n     И башни, и церковный свод;\r\n     Но не курится уж под ним\r\n     Кадильниц благовонный дым,\r\n     Не слышно пенье в поздний час\r\n     Молящих иноков за нас.\r\n     Теперь один старик седой,\r\n     Развалин страж полуживой,\r\n     Людьми и смертию забыт,\r\n     Сметает пыль с могильных плит,\r\n     Которых надпись говорит\r\n     О славе прошлой – и о том,\r\n     Как, удручен своим венцом,\r\n     Такой-т

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

М
и
х
а
и


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Михаил Юрьевич Лермонтов \r\nМцыри\r\n     \r\nБиблиотека школьника – \r\n     \r\n    Мцыри – на грузинском яз'
'ыке значит «неслужащий монах», нечто вроде «послушника». (Прим. Лермонтова.)\r\n    \r\n     \r\n    Вкушая'
', вкусих мало меда, и се аз умираю.\r\n    1-я Книга Царств\r\n     \r\n1\r\n     \r\n     Немного лет тому наз'
'ад,\r\n     Там, где, сливаяся, шумят,\r\n     Обнявшись, будто две сестры,\r\n     Струи Арагвы и Куры,\r\n '
'    Был монастырь. Из-за горы\r\n     И нынче видит пешеход\r\n     Столбы обрушенных ворот,\r\n     И башн'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Михаил Юрьевич Лермонтов \r\nМцыри\r\n     \r\nБиблиотека школьника – \r\n     \r\n    Мцыри – на грузинском я'
Target data: 'ихаил Юрьевич Лермонтов \r\nМцыри\r\n     \r\nБиблиотека школьника – \r\n     \r\n    Мцыри – на грузинском яз'


In [13]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 85) # (batch_size, sequence_length, vocab_size)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           21760     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 85)            87125     
                                                                 
Total params: 30,533,973
Trainable params: 30,533,973
No

In [19]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 85), dtype=float32, numpy=
array([[ 3.11225722e-06, -1.82862314e-05, -1.59145316e-06, ...,
        -1.54414720e-05, -1.43812485e-06,  1.40614375e-05],
       [ 3.31299816e-06, -4.42887758e-05, -5.53134396e-06, ...,
        -3.69139998e-05, -2.29976176e-05,  4.45472251e-05],
       [ 3.87774708e-06, -9.59575991e-05, -7.75303670e-06, ...,
        -7.43976707e-05, -6.45726104e-05,  9.57475859e-05],
       ...,
       [ 4.74811415e-04, -2.94006267e-03,  5.58431086e-04, ...,
        -2.10743444e-03, -1.95333664e-03,  3.90694058e-03],
       [ 7.34508678e-04, -2.98469630e-03,  7.67683028e-04, ...,
        -2.24083173e-03, -1.84607203e-03,  3.88215156e-03],
       [ 9.62522579e-04, -2.97567854e-03,  9.44981643e-04, ...,
        -2.34046322e-03, -1.74437137e-03,  3.85962473e-03]], dtype=float32)>

In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'тотчас!\r\n     Он с детских глаз уже не раз\r\n     Сгонял виденья снов живых\r\n     Про милых ближних и'

Next Char Predictions: 
 'ЕьУя,Зг)роА0и8ЛЯ(е\xa04!я«!?еьУь*Г\xa0м\r7Цч \n\nл0тмТ\xa0ГйЮкХГВжвзУ8(АбеЦ\rеКО7ыЗТг6?н,Ш,\xa0,\r99.\xa0ШГЯтжХ…щуНН7М*а'


In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 85)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.442816


In [23]:
model.compile(optimizer='adam', loss=loss)

In [28]:
!rm -rf ./training_checkpoints

In [29]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=7*10,
    save_weights_only=True)

In [32]:
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
7/7 [==============================] - 3s 353ms/step - loss: 0.0908
Epoch 2/100
7/7 [==============================] - 3s 352ms/step - loss: 0.0949
Epoch 3/100
7/7 [==============================] - 3s 359ms/step - loss: 0.0924
Epoch 4/100
7/7 [==============================] - 3s 360ms/step - loss: 0.0913
Epoch 5/100
7/7 [==============================] - 3s 359ms/step - loss: 0.0921
Epoch 6/100
7/7 [==============================] - 3s 365ms/step - loss: 0.0923
Epoch 7/100
7/7 [==============================] - 3s 369ms/step - loss: 0.0845
Epoch 8/100
7/7 [==============================] - 3s 370ms/step - loss: 0.0873
Epoch 9/100
7/7 [==============================] - 3s 371ms/step - loss: 0.0870
Epoch 10/100
7/7 [==============================] - 4s 531ms/step - loss: 0.0916
Epoch 11/100
7/7 [==============================] - 3s 374ms/step - loss: 0.0867
Epoch 12/100
7/7 [==============================] - 3s 373ms/step - loss: 0.0871
Epoch 13/100
7/7 [===================

In [33]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [34]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            21760     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 85)             87125     
                                                                 
Total params: 30,533,973
Trainable params: 30,533,973


In [36]:
def generate_text(model, start_string, temperature=1, num_generate=500):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = num_generate

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Продемонстрируем, как обучилась сеть. Возьмем достаточно большую температуру (подобрана эмпирически):

In [85]:
text_ = generate_text(model, start_string=u"Прекрасны горы под луной ", temperature=1.3)
print(text_)

Прекрасны горы под луной дал кой, будто он
     Лишь звуки дружеских имен
     Произносить был приучен.
     Простая песня то была,
     Но в мысль она мне залегла,
     И мн.
     Глубонил воткнуть
     И там два раза повернуть
     Мое оружье… Он засыл,
     Ких, нав? и скашал    Про легкий, бебеных коней,
     Про битвы чудные меж скал,
     Где всех один я побеждал!..
   гво гораю небес
     В, комля ды11
     
     Кругом меня цвел божий сад;
     Растений радужный наряд
     Хранил следы небесных 


Далее наиболее удачная температура для генерации - при ней модель соскакивает с одного зацикливания на другое:

In [86]:
text_ = generate_text(model, start_string=u"Прекрасны горы под луной ", temperature=1.1)
print(text_)

Прекрасны горы под луной дах, стачик слодил бежам и сиротой.
     
5
     
     Меня могила не страшит:
     Там, говорят, страданье спит
     В холодной вечной тишине;
     Но с жизнью жаль расстатам стених мне было далеко!
     Хотел я встать – пе ме     Узнать, для воли иль тюрьмы
     Не этот свет родимся мы.
     И в час ночной, ужасный час,
     Когда гроза п   И жаждут встречи каждый миг;
     Но дни бегут, бегут года –
     Им не сойтиться никогда!
     Я вител горные хребты,
     Причудливые, как


В этом тексте угадываются несколько реальных отрывков, соединенных неудачной генерацией.

Далее ниже 0.9 модель скатывается в одинаковую генерацию большого длинного отрывка:

In [87]:
text_ = generate_text(model, start_string=u"Прекрасны горы под луной ", temperature=.1)
print(text_)

Прекрасны горы под луной дад коста,
     И толь в прокладных страных стенах
     Душой дитя, судьбой монах.
     Я никому не мог сказать
     Священных слов «отец» и «мать».
     Конечно, ты хотел, старик,
     Чтоб я в обители отвык
     От этих сладостных имен, –
     Напрасно: звук их был рожден
     Со мной. И видел у других
     Отчизну, дом, друзей, родных,
     А у себя не находил
     Не только милых душ – могил!
     Тогда, пустых не тратя слез,
     В душе я клятву произнес:
     Хотя на миг ког


In [88]:
text_ = generate_text(model, start_string=u"Прекрасны горы под луной ", temperature=.01)
print(text_)

Прекрасны горы под луной дад коста,
     И толь в прокладных страных стенах
     Душой дитя, судьбой монах.
     Я никому не мог сказать
     Священных слов «отец» и «мать».
     Конечно, ты хотел, старик,
     Чтоб я в обители отвык
     От этих сладостных имен, –
     Напрасно: звук их был рожден
     Со мной. И видел у других
     Отчизну, дом, друзей, родных,
     А у себя не находил
     Не только милых душ – могил!
     Тогда, пустых не тратя слез,
     В душе я клятву произнес:
     Хотя на миг ког


In [38]:
len(text_)

515

**Выводы по генерации**
- Модель воспроизводит длинную последовательность.
- Модель вместо составления смешных предложений из отдельных слов сваливается в воспроизведение нескольких или одного отрывка из произведения. Тут, наверное, проблема в том, что Мцыри - довольно короткое произведение, и разброс различных сочетаний букв (и слов тоже) у него ограничен. Поэтому модель просто переобучилась на отдельные участки текста и воспроизводит их.
- Разные начальные строки дают сваливаться в воспроизведение разных участков произведения. Но смотреть на этот процесс надо следующим образом: разные начальные строки генерируют разный первый начальный символ. А уже с этого начального символа модель воспроизводит определенный участок текста.
- Пробовал изменить функцию генерации, после первого предикта, подавая снова на вход не один символ, а входную строку + первый символ. Но качество генерации это не изменило, а воспроизведение одного участка текста даже усилилось. Так как при генерации из одного символа уровень неопределонности предсказания следующего символа выше, чем если следующий символ предсказывается от целой строки.